Installazione delle Librerie Necessarie

In [3]:
pip install astroquery pandas matplotlib seaborn ipython-sql sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [4]:
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

# Carica l'estensione ipython-sql
%load_ext sql


In [5]:
def scarica_e_salva_dati():
    """
    Scarica i dati degli esopianeti confermati dal NASA Exoplanet Archive usando una query SQL
    e li salva in un database SQLite.
    """
    print("Scaricamento dei dati dal NASA Exoplanet Archive tramite SQL...")
    try:
        query = """
        SELECT pl_name, pl_bmasse, pl_rade, pl_orbsmax, pl_eqt, st_spectype
        FROM ps
        WHERE pl_bmasse IS NOT NULL
          AND pl_rade IS NOT NULL
          AND pl_orbsmax IS NOT NULL
          AND pl_eqt IS NOT NULL
          AND st_spectype IS NOT NULL
        """
        dati = NasaExoplanetArchive.query_criteria(query=query, table="ps")
        df = dati.to_pandas()
        print("Download completato. Numero di esopianeti scaricati:", len(df))
        
        # Salva i dati in un database SQLite
        conn = sqlite3.connect('esopianeti.db')
        df.to_sql('esopianeti', conn, if_exists='replace', index=False)
        conn.close()
        print("Dati salvati nel database SQLite 'esopianeti.db'.")
    except Exception as e:
        print("Errore durante il download o il salvataggio dei dati:", e)

# Esegui la funzione per scaricare e salvare i dati
# scarica_e_salva_dati()


In [6]:
# Connessione al database SQLite
%sql sqlite:///esopianeti.db


In [7]:
%%sql
-- Esempio: Seleziona i primi 5 esopianeti
SELECT * FROM esopianeti LIMIT 5;


 * sqlite:///esopianeti.db
(sqlite3.OperationalError) no such table: esopianeti
[SQL: -- Esempio: Seleziona i primi 5 esopianeti
SELECT * FROM esopianeti LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [8]:
%%sql
-- Esempio: Seleziona esopianeti con massa superiore a 10 masse terrestri
SELECT pl_name, pl_bmasse, pl_rade
FROM esopianeti
WHERE pl_bmasse > 10;


 * sqlite:///esopianeti.db
(sqlite3.OperationalError) no such table: esopianeti
[SQL: -- Esempio: Seleziona esopianeti con massa superiore a 10 masse terrestri
SELECT pl_name, pl_bmasse, pl_rade
FROM esopianeti
WHERE pl_bmasse > 10;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [9]:
# Connessione al database SQLite con pandas
conn = sqlite3.connect('esopianeti.db')

# Esempio: Carica tutti i dati in un DataFrame
df_pulito = pd.read_sql_query("SELECT * FROM esopianeti", conn)
conn.close()

# Visualizza le prime 5 righe
df_pulito.head()


DatabaseError: Execution failed on sql 'SELECT * FROM esopianeti': no such table: esopianeti

In [ ]:
def pulisci_dati(df):
    """
    Pulisce e filtra il DataFrame per i campi di interesse.
    """
    print("Inizio della pulizia e del filtraggio dei dati...")
    # Ad esempio, rimuovi esopianeti con temperature estremamente alte o basse
    df_pulito = df[(df['pl_eqt'] > 200) & (df['pl_eqt'] < 3000)]
    print("Dati puliti. Numero di esopianeti dopo il filtraggio:", len(df_pulito))
    return df_pulito

# Pulisci i dati
df_pulito = pulisci_dati(df_pulito)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_massa_vs_raggio(df):
    """
    Crea un grafico scatter di Massa vs Raggio e calcola la correlazione.
    """
    plt.figure(figsize=(10,6))
    sns.scatterplot(data=df, x='pl_bmasse', y='pl_rade', alpha=0.6)
    plt.title('Massa vs Raggio degli Esopianeti')
    plt.xlabel('Massa (Massa Terra)')
    plt.ylabel('Raggio (Raggio Terra)')
    
    # Calcolo della correlazione
    correlazione = df['pl_bmasse'].corr(df['pl_rade'])
    plt.annotate(f'Correlazione: {correlazione:.2f}', xy=(0.05, 0.95), xycoords='axes fraction',
                 fontsize=12, ha='left', va='top')
    plt.show()

def plot_temp_vs_distanza(df):
    """
    Crea un grafico scatter della Temperatura vs Distanza Orbitale.
    """
    plt.figure(figsize=(10,6))
    sns.scatterplot(data=df, x='pl_orbsmax', y='pl_eqt', alpha=0.6)
    plt.title('Temperatura di Equilibrio vs Distanza Orbitale')
    plt.xlabel('Distanza Orbitale (AU)')
    plt.ylabel('Temperatura di Equilibrio (K)')
    plt.show()

# Esempio di utilizzo
# plot_massa_vs_raggio(df_pulito)
# plot_temp_vs_distanza(df_pulito)


In [ ]:
def menu_interattivo_sql():
    """
    Menu interattivo per pilotare l'analisi dei dati utilizzando SQL.
    """
    conn = sqlite3.connect('esopianeti.db')
    while True:
        print("\n=== Analisi del Catalogo di Esopianeti (Con SQL) ===")
        print("1. Visualizza i primi 5 record dei dati")
        print("2. Seleziona esopianeti con massa superiore a un valore")
        print("3. Grafico Massa vs Raggio")
        print("4. Grafico Temperatura vs Distanza Orbitale")
        print("5. Esci")
        
        scelta = input("Inserisci il numero dell'operazione che desideri eseguire: ")
        
        if scelta == '1':
            query = "SELECT * FROM esopianeti LIMIT 5;"
            df = pd.read_sql_query(query, conn)
            print("\nPrime 5 righe del DataFrame pulito:")
            print(df)
        elif scelta == '2':
            valore = input("Inserisci il valore minimo della massa (Massa Terra): ")
            try:
                valore = float(valore)
                query = f"""
                SELECT pl_name, pl_bmasse, pl_rade
                FROM esopianeti
                WHERE pl_bmasse > {valore};
                """
                df = pd.read_sql_query(query, conn)
                print(f"\nEsopianeti con massa superiore a {valore} Masse Terra:")
                print(df)
            except ValueError:
                print("Valore inserito non valido. Inserisci un numero.")
        elif scelta == '3':
            plot_massa_vs_raggio(df_pulito)
        elif scelta == '4':
            plot_temp_vs_distanza(df_pulito)
        elif scelta == '5':
            print("Uscita dal programma. Arrivederci!")
            break
        else:
            print("Scelta non valida. Riprova.")
    conn.close()

# Avvia il menu interattivo
# menu_interattivo_sql()
